In [ ]:
import numpy
import urllib.request
import warnings

# module-level imports
from astropy import (coordinates, units)
from astropy.io import fits

In [ ]:
# matplotlib imports
import matplotlib.pyplot as plt
from matplotlib import patheffects
%matplotlib inline

In [ ]:
# global variables
CMAP = 'bone_r'
BATSE_PATH = ('https://gammaray.nsstc.nasa.gov/'
              'batse/grb/catalog/current/tables')
BATSE_BASIC = '/'.join((BATSE_PATH, 'basic_table.txt'))
BATSE_FLUX = '/'.join((BATSE_PATH, 'flux_table.txt'))
BATSE_DURATION = '/'.join((BATSE_PATH, 'duration_table.txt'))
HALPHA_SURVEY = ('https://lambda.gsfc.nasa.gov/data/foregrounds/halpha/'
                 'images/lambda_mollweide_halpha_fwhm06_0512.fits')

In [ ]:
warnings.filterwarnings('ignore', category=RuntimeWarning)

In [ ]:
# -- functions ----------------------------------------------------------------

def read_txt_from_url(path, columns, start=1, step=1):
    """Reads a .txt file given a URL, and returns the data as a collection of
    masked numpy arrays

    Parameters
    ----------
    path : `str`
        URL of the file to read

    columns : `tuple` of `int`
        columns to read from the file, should be a `tuple` of `int` for
        multiple columns

    start : `int`, optional
        line of the file to start on, default: 0

    step : `int`, optional
        number of lines to step at a time, default: 1

    Returns
    -------
    data : `list` of `numpy.ndarray`
        requested column data from the file
    """
    # request HTTP access to the given file
    with urllib.request.urlopen(path) as response:
        lines = response.readlines()[start-1::step]
    data = [numpy.array(  # re-pack requested rows and columns
        [float(line.split()[i-1]) for line in lines]
    ) for i in columns]
    return data

In [ ]:
# read the galactic longitude, galactic latitude, and error radius
# from columns 8, 9, and 10 of the BATSE 4b basic catalogue
(lon, lat, err) = read_txt_from_url(BATSE_BASIC, columns=(8, 9, 10))
lon = coordinates.Angle((lon + 180) * units.degree)
lat = coordinates.Angle(lat * units.degree)
err = coordinates.Angle(err * units.degree)

# set zero-point of longitude at 180 degrees
lon.wrap_at(180 * units.degree, inplace=True)

In [ ]:
# download H-alpha survey data from NASA
with urllib.request.urlopen(HALPHA_SURVEY) as response:
    (_, milkyway) = fits.open(response)

In [ ]:
fig = plt.figure(figsize=(12, 6))
ax1 = fig.gca()
ax1.axis('off')

# show a map of the Milky Way
im = ax1.imshow(milkyway.data, origin='lower', cmap=CMAP)
im.set_clim(0, 30)

# draw a new set of axes on top of the Milky Way map
ax2 = fig.add_axes(ax1.get_position(), projection='mollweide', frameon=False)

# draw the location of each BATSE burst, with detector's localization error
ax2.scatter(lon.radian, lat.radian, s=20*err.degree, marker='o',
            alpha=0.4, edgecolor='#222222', facecolor='#4ba6ff')

# make easier-to-read axis labels
labels = ax2.set_xticklabels([
    '330$^{\degree}$', '300$^{\degree}$', '270$^{\degree}$', '240$^{\degree}$',
    '210$^{\degree}$', '180$^{\degree}$', '150$^{\degree}$', '120$^{\degree}$',
    '90$^{\degree}$', '60$^{\degree}$', '30$^{\degree}$'])
for l in labels:
    l.set_path_effects([
        patheffects.Stroke(
            linewidth=4,
            foreground=plt.cm.get_cmap(CMAP)(0)),
        patheffects.Normal()])
plt.show()

In [ ]:
# read in BATSE flux data, then sort in ascending order
(flux, ) = read_txt_from_url(
    BATSE_FLUX,
    columns=(1,),
    start=4,
    step=5,
)
flux.sort()

# prepare a figure
fig = plt.figure(figsize=(12, 6))
ax = fig.gca()

# plot a cumulative histogram of log(flux)
plt.hist(flux, bins=flux, histtype='step', color='#4ba6ff',
         linewidth=2, cumulative=-1, label='BATSE 4B catalogue')

# plot the uniform volume distribution
# expected from Euclidean universe
S = numpy.logspace(-1, 3, 101)
N = 10 * (S / 55) ** (-3/2)
ax.plot(S, N, 'k--', linewidth=1.25, label='$S^{-3/2}$')

# x-axis features
ax.set_xscale('log')
ax.set_xlim([1, flux.max()])
ax.set_xlabel('Peak flux [photons cm$^{-2}$ s$^{-1}$]')
ax.set_xticks([1, 10, 100])
ax.set_xticklabels(['1', '10', '100'])

# y-axis features
ax.set_yscale('log')
ax.set_ylim([1, 1e3])
ax.set_ylabel('Number of GRBs')
ax.set_yticks([1, 10, 100, 1000])
ax.set_yticklabels(['1', '10', '100', '$10^3$'])

# draw a legend and show
plt.legend(loc='best')
plt.show()

In [ ]:
# read in BATSE duration data
(T90, ) = read_txt_from_url(BATSE_DURATION, columns=(5,))

# prepare a figure
fig = plt.figure(figsize=(12, 6))
ax = fig.gca()

# plot a histogram of log(T90)
bins = numpy.logspace(-3, 3, 31)
ax.hist(T90, bins=bins, histtype='step', color='#222222', linewidth=2)
ax.plot([2, 2], [0, 300], 'k--', linewidth=1)

# x-axis features
ax.set_xscale('log')
ax.set_xlim([1e-3, 1e3])
ax.set_xlabel('$T_{90}$ [seconds]')

# y-axis features
ax.set_ylim([0, 300])
ax.set_ylabel('Number of GRBs')

# show the figure
plt.show()